In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.x = None
    
    def forward(self,x,y):
        self.x = x 
        self.y = y
        out = x * y        

        return out
    
    def backward(self, dout):
        dx = dout * self.y 
        dy = dout * self.x
        
        return dx, dy
        

apple = 100
apple_num = 2 
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [4]:
dprice = 1 
dapple_price , dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [7]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        
        return  out
    
    def backward(self, dout):
        dx = dout*1 
        dy = dout*1
        
        return dx, dy


In [10]:
apple_num = 2 
apple_price = 100
orange_num = 3
orange_price = 150
tax = 1.1

mul_apple_price = MulLayer()
mul_orange_price = MulLayer()
add_orange_apple = AddLayer()
mul_orange_apple_tax = MulLayer()

apple_price = mul_apple_price.forward(apple_price, apple_num)
orange_price = mul_orange_price.forward(orange_price, orange_num)
orange_apple = add_orange_apple.forward(apple_price, orange_price)
orange_apple_tax = mul_orange_apple_tax.forward(orange_apple,tax)
price = orange_apple_tax

dprice = 1

dorange_apple, dtax = mul_orange_apple_tax.backward(dprice)
dapple_price, dorange_price = add_orange_apple.backward(dorange_apple)
dapple, dapple_num = mul_apple_price.backward(dapple_price)
dorange, dorange_num = mul_orange_price.backward(dorange_price)


In [11]:
print(price)
print(dapple_num, dapple,dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [12]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [17]:
import numpy as np
x = np.array([[1.0,-0.5],[-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

x[mask] = 0
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
[[1. 0.]
 [0. 3.]]


In [18]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out 
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx 

In [21]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W) + B 
print(Y.shape)

(2,)
(2, 3)
(3,)
(3,)


In [22]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [23]:
dY = np.array([[1,2,3],[4,5,6]])
print(dY)

dB = np.sum(dY, axis = 0)
print(dB)


[[1 2 3]
 [4 5 6]]
[5 7 9]


In [25]:
class Affine:
    def __init__(self, W, b):
        self.W = W 
        self.b = b 
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x 
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx
        
        

In [24]:
a = np.array([[1,2,3],[4,5,6]])
print(a)
print(a.T)

[[1 2 3]
 [4 5 6]]
[[1 4]
 [2 5]
 [3 6]]


In [18]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

from common.layers import * 
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std =0.01):
        
        self.params = {}
        self.params['W1'] = weight_init_std *  np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std *  np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.listLayer = SoftmaxWithLoss()
            
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)   
            
        return x 

    # x: 입력 데이터, t : 정답 레이블 
    def loss(self, x, t):
        y = self.predict(x)
        return self.listLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis= 1 )
        if t.ndim != 1 : t = np.argmax(t, axis= 1)

        accuracy = np.sum(y == t) / float(x.shape[0])

        return accuracy

    # x: 입력 데이터, t : 정답 레이블 
    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x,t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        #순전파 
        self.loss(x, t)

        #역전파
        dout = 1
        dout = self.listLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        #결과 저장 
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads



### 5.7.3 오차역전파법으로 구한 기울기 검증하기 


In [19]:
import sys, os
sys.path.append(os.pardir)
import numpy as np 
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

# 데이터 읽기 
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size= 784, hidden_size = 50, output_size=10 )

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop  = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key]-grad_numerical[key]))
    print(key +':'+ str(diff))

W1:3.9255895844648973e-10
b1:2.308228310113781e-09
W2:6.186535624278964e-09
b2:1.3954629068113621e-07


In [20]:
import sys, os
sys.path.append(os.pardir)
import numpy as np 
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet



In [23]:
import numpy as np 
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize= True, one_hot_label= True)

network = TwoLayerNet(input_size= 784, hidden_size= 50, output_size=10)


iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/ batch_size, 1)


for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        print('train acc, test acc |' + str(train_acc) + ', ' + str(test_acc))


train acc, test acc |0.09846666666666666, 0.1004
train acc, test acc |0.9041666666666667, 0.9049
train acc, test acc |0.9247, 0.9284
train acc, test acc |0.9344666666666667, 0.9355
train acc, test acc |0.9426666666666667, 0.9446
train acc, test acc |0.9501833333333334, 0.9497
train acc, test acc |0.9568, 0.9548
train acc, test acc |0.96115, 0.9578
train acc, test acc |0.9634333333333334, 0.959
train acc, test acc |0.9659333333333333, 0.9588
train acc, test acc |0.9666833333333333, 0.9633
train acc, test acc |0.9711333333333333, 0.9656
train acc, test acc |0.9726833333333333, 0.9644
train acc, test acc |0.9724, 0.9652
train acc, test acc |0.9752833333333333, 0.9673
train acc, test acc |0.9776, 0.9689
train acc, test acc |0.97705, 0.9696
